In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint
from pathlib import Path
from random import randint

import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import nibabel as nib
import glmsingle
from glmsingle.glmsingle import GLM_single
import bids
from bids import BIDSLayout
from scipy.ndimage import zoom, binary_dilation
import h5py
import nibabel as nib
from einops import rearrange

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from tc2see import load_data

In [ ]:
dataset_root = Path('E:\\fmri_processing\\results')
tc2see_version = 1 # [1, 2]
#dataset_path = dataset_root / f"TC2See_v{tc2see_version}"
dataset_path = dataset_root
derivatives_path = dataset_path / 'derivatives_TC2See_prdgm'

In [ ]:
tr = 1.97
bold, stimulus_ids, mask, affine = load_data(
    derivatives_path / 'tc2see-v3-bold.hdf5', 
    'sub-03', 
    tr_offset=4 / tr,
    run_normalize='linear_trend',
    interpolation=False
)
bold.shape

In [ ]:
nc_file = 'noise_ceiling/sub-01/sub-01__time-linspace_0_6_13__interpolation-false__noise-ceiling.nii.gz'
nc_series = nib.load(derivatives_path / nc_file).get_fdata()
nc_volume = nc_series[..., 8]
nc_volume[:, :, :5] = 0.
nc = nc_volume[mask]
nc.shape

nc_threshold = 25.
X = bold[:, nc > nc_threshold]

In [ ]:
stimulus_images = h5py.File(derivatives_path / 'stimulus-images.hdf5', "r")
image_names = list(stimulus_images.keys())
image_names = [image_names[i] for i in stimulus_ids]
image_names = np.array(image_names)

images = [stimulus_images[name]['data'][:] for name in image_names]
bird_class = np.array([stimulus_images[name].attrs['class_id'] for name in image_names])
bird_superclass = (bird_class > 131).astype(int)

Y = np.unique(bird_class, return_inverse=True)[1]
#Y = bird_superclass

ids = np.arange(Y.shape[0])
np.random.shuffle(ids)
Y = Y[ids]

In [ ]:
Y_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import KFold

folds = KFold(n_splits=5, shuffle=True, random_state=0)
Y_pred = np.zeros_like(Y)
for train_ids, val_ids in tqdm(folds.split(X)):
    X_train, Y_train = X[train_ids], Y[train_ids]
    X_val, Y_val = X[val_ids], Y[val_ids]
    
    #model = LogisticRegression(solver='lbfgs', max_iter=1000)
    model = RidgeClassifier(alpha=1000)
    model.fit(X_train, Y_train)
    Y_pred[val_ids] = model.predict(X_val)
    
from sklearn import metrics
metrics.accuracy_score(Y, Y_pred)

In [ ]:
model.score(Y, Y_pred)

In [ ]:
Y_pred.shape

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
A, B = load_iris(return_X_y=True)
A.shape, B.shape
B